# 💬 LangGraph Chatbot: Basic Implementation

Build a simple conversational AI agent using LangGraph and Google Gemini.

## 🎯 What You'll Learn

1. **Chatbot State** - Managing conversation history
2. **add_messages** - Automatic message appending
3. **Simple Graph Structure** - Linear chatbot flow
4. **LLM Integration** - Connecting Google Gemini
5. **Interactive Loop** - Continuous conversation

## ✨ What This Chatbot Does

- ✅ Takes user input
- ✅ Sends to Google Gemini LLM
- ✅ Returns AI response
- ✅ Maintains conversation context
- ✅ Runs in a continuous loop

**Note:** This is a basic chatbot WITHOUT tools. It only does text conversation.

---

## 📦 Step 1: Import Dependencies

In [ ]:
# LangGraph imports
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END

# LangChain imports
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

# Environment
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv()

print("✅ Dependencies imported successfully!")

{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='183052e0-9f16-43e9-b05d-aad67969d9ea'), AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--5d81eefd-603a-4c60-9a82-717bdccb3188-0', usage_metadata={'input_tokens': 2, 'output_tokens': 32, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 23}})]}


## 🧠 Step 2: Initialize LLM

In [ ]:
# Initialize Google Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

print("✅ Gemini 2.5 Flash model initialized!")

## 🏗️ Step 3: Define State

**Key Feature:** `add_messages` reducer
- Automatically appends new messages to the list
- Maintains full conversation history
- No manual list management needed

In [ ]:
# Define chatbot state with message history
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]
    # ↑ This annotation means:
    # - messages is a list
    # - add_messages automatically appends new messages
    # - No need to manually manage message history

print("✅ State schema defined with automatic message handling!")

## ⚙️ Step 4: Define Chatbot Node

The node function:
1. Receives current state (with message history)
2. Sends all messages to the LLM
3. Returns the LLM's response
4. State automatically merges the new message

In [ ]:
def chatbot(state: BasicChatState):
    """
    Chatbot node:
    - Takes state with message history
    - Invokes LLM with all messages (for context)
    - Returns new AI message
    """
    return {
        "messages": [llm.invoke(state["messages"])]
    }

print("✅ Chatbot node function defined!")

## 🔧 Step 5: Build the Graph

Simple linear flow:
```
START → chatbot → END
```

In [ ]:
# Create graph
graph = StateGraph(BasicChatState)

# Add chatbot node
graph.add_node("chatbot", chatbot)

# Set entry point
graph.set_entry_point("chatbot")

# Add edge to END (conversation ends after each response)
graph.add_edge("chatbot", END)

# Compile
app = graph.compile()

print("✅ Graph compiled!")
print("\n📊 Graph Structure:")
print("   START → chatbot → END")

## 💬 Step 6: Run the Chat Loop

Interactive conversation loop:
- Takes user input
- Creates HumanMessage
- Invokes graph
- Prints full response

**Type "exit" or "end" to quit**

In [ ]:
print("🤖 Chatbot started! Type 'exit' or 'end' to quit.\n")

while True: 
    user_input = input("You: ")
    
    # Check for exit commands
    if user_input.lower() in ["exit", "end"]:
        print("👋 Goodbye!")
        break
    else: 
        # Run the graph with user message
        result = app.invoke({
            "messages": [HumanMessage(content=user_input)]
        })
        
        # Print the full result (shows all messages)
        print(f"\nAI: {result['messages'][-1].content}\n")

---

## 🎓 Key Concepts Explained

### 1️⃣ **add_messages Reducer**

```python
messages: Annotated[list, add_messages]
```

This special annotation automatically handles message history:
- **Without it**: You'd need to manually append each message
- **With it**: New messages are automatically added to the list
- **Benefits**: Cleaner code, less error-prone

### 2️⃣ **State Flow**

```
User Input → HumanMessage → Graph Invoke → LLM Processing → AI Message → State Update
```

Each invoke:
1. Adds HumanMessage to state
2. Chatbot node processes all messages
3. Returns AIMessage
4. State automatically merges new message
5. Full history preserved for next turn

### 3️⃣ **Why This Pattern?**

| Aspect | Benefit |
|--------|---------|
| **State Persistence** | Conversation context maintained |
| **Automatic Merging** | No manual list operations |
| **Clean Code** | Single node handles all logic |
| **Scalable** | Easy to add more nodes later |

---

## 🚀 Next Steps

This basic chatbot can be extended with:

1. **Tools** - Add web search, calculations, etc.
2. **Memory** - Persist conversations across sessions
3. **Conditional Routing** - Different behaviors based on input
4. **Human-in-the-Loop** - Approval workflows
5. **Multi-Agent** - Multiple specialized chatbots

Check out `chatbotwith_tools.ipynb` for the next level! 🎯